In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation

def I(x):
    L = x[-1]
    len_x = np.size(x)
    i_x = np.zeros(len_x)
    for i in range(len_x):
        if x[i] < (L+1)/2 and x[i] > (L-1)/2:
            i_x[i] = 1
    return i_x

def U_exact(x, t):
    L = 5
    u_ex = np.ones(np.size(x)) * (1/L)
    
    for n in range(1, 2000):
        c_n = (2 / (n * np.pi)) * (np.sin((n * np.pi * (L + 1)) / (2 * L)) - np.sin((n * np.pi * (L - 1)) / (2 * L)))
        
        u_ex += c_n * np.cos(n*np.pi*x / L)*np.exp(-1*n** 2 * np.pi**2 * t / L **2)
        
    return u_ex

In [ ]:
L=5; T=2
a = 1

Nx_spaces = 200; Nt_gaps = 100; 
Nx_points = Nx_spaces +  1 ; Nt_points = Nt_gaps + 1

x = np.linspace(0, L, Nx_points)   # mesh points in space
dx = x[1] - x[0]

t = np.linspace(0, T, Nt_points) # mesh points in time
dt = t[1] - t[0]

C = a * dt/dx**2
print("dx=",dx,"dt=", dt,"C =",C)

# set up matrices
u   = np.zeros(Nx_points)
u_old = np.zeros(Nx_points)

U = np.zeros((Nx_points,Nt_points))
U_ex = np.zeros((Nx_points, Nt_points))

# Data structures for the linear system
A = np.zeros((Nx_points, Nx_points))
b = np.zeros(Nx_points)

# set up the matrix A
for i in range(1, Nx_points-1): # rows from 1 to Nx-2
    A[i,i-1] = -C
    A[i,i+1] = -C
    A[i,i] = 1 + 2*C
A[0,0] = 1+2*C  ; A[0,1]= -2*C 
A[Nx_points-1,Nx_points-1] = 1+2*C 
A[Nx_points-1,Nx_points-2] = -2*C

# find the inverse of A and use for all points
Ainv = np.linalg.inv(A)

# Set initial condition u(x,0) = I(x)
u_old = I(x) # no dirichlet boundary conditions in this example

# initialise matrices U and U_ex2 for first time step
U[:,0] = u_old[:]
U_ex[:,0] = U_exact(x,0)

#perform time=stepping
for n in range(1, Nt_points): # timestep for 1 to t = T-1 so last step finishes on t=T
    # Compute b and solve linear system
    b[:] = u_old[:]
    u[:] = np.dot(Ainv,b)
    # Update u_1 before next step
    u_old = u
    U[:,n] = u
    U_ex[:,n]=U_exact(x,t[n])

In [ ]:
# set up animation plots
fig, ax = plt.subplots()
ax.axis([-1,6,0,1])
def animate(i):
    l.set_data(x, U[:,i])
    m.set_data(x,U_ex[:,i])
    
ax.axis([-1,6,0,1])
l, = ax.plot([],[],':r')
m, = ax.plot([],[],'-.b')

ani2 = matplotlib.animation.FuncAnimation(fig, animate, frames=Nt_points)

from IPython.display import HTML
HTML(ani2.to_jshtml())

In [ ]:
times = np.linspace(0, Nt_gaps, 4, dtype = int)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize= (10,10))


ax[0, 0].axis([-1, 6, 0, 1.5])
ax[1, 0].axis([-1, 6, 0, 1.5])
ax[0, 1].axis([-1, 6, 0, 1.5])
ax[1, 1].axis([-1, 6, 0, 1.5])

ax[0, 0].plot(x, U[:, times[0]], ':c', label = 'numerical')
ax[0, 0].plot(x, U_ex[:, times[0]], '-.m', label = 'exact')
ax[0, 0].set_title(f'time = {t[times[0]]}')

ax[0, 1].plot(x, U[:, times[1]], ':c')
ax[0, 1].plot(x, U_ex[:, times[1]], '-.m')
ax[0, 1].set_title(f'time = {t[times[1]]}')

ax[1, 0].plot(x, U[:, times[2]], ':c')
ax[1, 0].plot(x, U_ex[:, times[2]], '-.m')
ax[1, 0].set_title(f'time = {t[times[2]]}')

ax[1, 1].plot(x, U[:, times[3]], ':c')
ax[1, 1].plot(x, U_ex[:, times[3]], '-.m')
ax[1, 1].set_title(f'time = {t[times[3]]}')


fig.legend(bbox_to_anchor= (1.2, 0.9))
plt.tight_layout()